In [4]:
import os

import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image


from tensorflow.keras.layers import Flatten, Dense, Input, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
import keras
from keras import layers, models

from keras.applications.vgg16 import VGG16

from sklearn.preprocessing import LabelBinarizer


KeyboardInterrupt



In [1]:
TRAIN_DATA_PATH = '../data/traffic_data_old/Train'
TRAIN_CSV = '../data/traffic_data_old/Train.csv'
TEST_CSV = '../data/traffic_data_old/Test.csv'
TEST_DATA_PATH = '../data/traffic_data_old'
MODEL_PATH = '../data/traffic_data_model_2.h5'
# SINGLE_OBJ_MODEL_PATH = '/content/single_obj_model.h5'
CLASSES = 43

# hyper-parameters
# LEARNING_RATE = 1e-4
# EPOCHS = 25
# BATCH_SIZE = 32
sample_size = 90
test_size = int(sample_size*43*0.2)

In [ ]:
print(test_size)

688


In [2]:
train_csv_data = pd.read_csv(TRAIN_CSV)
new_train_data = train_csv_data[train_csv_data.ClassId == 0].sample(n=sample_size)

for i in range(1, 43):
    cur_data = train_csv_data[train_csv_data.ClassId == i].sample(n=sample_size)
    new_train_data = [new_train_data, cur_data]
    new_train_data = pd.concat(new_train_data)


NameError: name 'pd' is not defined

In [ ]:
%%time
train_data = [] # image data
train_targets = [] # our predictions
train_classIds = []
train_filenames = []
for ind in new_train_data.index:
    img = Image.open(TRAIN_DATA_PATH + '/../' + new_train_data.Path[ind])
    img = img.resize((224, 224))
    img = np.array(img)

    train_data.append(img)

    x1_ = new_train_data['Roi.X1'][ind] / new_train_data.Width[ind]
    y1_ = new_train_data['Roi.Y1'][ind] / new_train_data.Height[ind]
    x2_ = new_train_data['Roi.X2'][ind] / new_train_data.Width[ind]
    y2_ = new_train_data['Roi.Y2'][ind] / new_train_data.Height[ind]
    train_targets.append((x1_, y1_, x2_, y2_))

    train_classIds.append(new_train_data.ClassId[ind])
    train_filenames.append(new_train_data.Path[ind])

In [3]:
%%time
train_data = np.array(train_data, dtype='float') / 255.0
train_targets = np.array(train_targets, dtype='float')
train_filenames = np.array(train_filenames)
train_classIds = np.array(train_classIds)

lb = LabelBinarizer()
train_classIds = lb.fit_transform(train_classIds)

NameError: name 'np' is not defined

In [11]:
test_csv_data = pd.read_csv(TEST_CSV)
new_test_data = test_csv_data.sample(n=test_size)

In [12]:
%%time
test_data = [] # image data
test_targets = [] # our predictions
test_classIds = []
test_filenames = []
for ind in new_test_data.index:
    img = Image.open(TEST_DATA_PATH + '/../' + new_test_data.Path[ind])
    img = img.resize((224, 224))
    img = np.array(img)

    test_data.append(img)

    x1_ = new_test_data['Roi.X1'][ind] / new_test_data.Width[ind]
    y1_ = new_test_data['Roi.Y1'][ind] / new_test_data.Height[ind]
    x2_ = new_test_data['Roi.X2'][ind] / new_test_data.Width[ind]
    y2_ = new_test_data['Roi.Y2'][ind] / new_test_data.Height[ind]
    test_targets.append((x1_, y1_, x2_, y2_))

    test_classIds.append(new_test_data.ClassId[ind])
    test_filenames.append(new_test_data.Path[ind])

CPU times: user 822 ms, sys: 22.9 ms, total: 845 ms
Wall time: 843 ms


In [12]:
# new_test_data[new_test_data.ClassId == 42]
# test_classIds.append(43)

In [13]:
%%time
test_data = np.array(test_data, dtype='float') / 255.0
test_targets = np.array(test_targets, dtype='float')
test_filenames = np.array(test_filenames)
test_classIds = np.array(test_classIds)

test_classIds = lb.fit_transform(test_classIds)
# test_classIds = test_classIds[:-1]

CPU times: user 130 ms, sys: 190 ms, total: 321 ms
Wall time: 323 ms


In [ ]:
print(len(test_classIds[0]))

43


In [14]:
# loading model
multi_vgg = VGG16(weights='imagenet', include_top=False, input_tensor=Input(shape=(224, 224, 3)))

# make layer untrainable
multi_vgg.trainable = False

# flattening output layer
flatten = multi_vgg.output
flatten_layer = Flatten()(flatten)

# fully connected layer to predict bounding boxes
fc1 = Dense(128, activation='relu')(flatten_layer)
fc1 = Dense(64, activation='relu')(fc1)
fc1 = Dense(32, activation='relu')(fc1)
fc1 = Dense(4, activation='sigmoid', name='box_coodinates')(fc1)
output1 = fc1

# fully connected layer to predict classes
fc2 = Dense(512, activation='relu')(flatten_layer)
fc2 = Dropout(0.5)(fc2)
fc2 = Dense(512, activation='relu')(fc2)
fc2 = Dropout(0.5)(fc2)
fc2 = Dense(len(lb.classes_), activation='softmax', name='class_label')(fc2)
output2 = fc2

model = Model(inputs=multi_vgg.input, outputs=(fc1, fc2))

58900480/58889256 [==============================] - 0s 0us/step


In [15]:
# initialising loses and optimizer
losses = {
    'class_label': 'categorical_crossentropy',
    'box_coodinates': 'mean_squared_error'
}

lossWeights = {
    "class_label": 1.0,
    "box_coodinates": 1.0
}

opt = Adam(learning_rate=LEARNING_RATE)
model.compile(loss=losses, optimizer=opt, metrics=['accuracy'], loss_weights=lossWeights)
print(model.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 224, 224, 64  1792        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv2 (Conv2D)          (None, 224, 224, 64  36928       ['block1_conv1[0][0]']           
                                )                                                             

In [16]:
train_outputs = {
    'class_label': train_classIds,
    'box_coodinates': train_targets
}

test_outputs = {
    'class_label': test_classIds,
    'box_coodinates': test_targets
}

In [17]:
print(len(test_outputs['class_label'][0]))

43


In [18]:
history = model.fit(train_data, train_outputs, validation_data=(test_data, test_outputs), batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=1)

Epoch 1/25
121/121 [==============================] - 42s 224ms/step - loss: 3.6676 - box_coodinates_loss: 0.0054 - class_label_loss: 3.6622 - box_coodinates_accuracy: 0.5282 - class_label_accuracy: 0.0646 - val_loss: 3.4026 - val_box_coodinates_loss: 6.1064e-04 - val_class_label_loss: 3.4020 - val_box_coodinates_accuracy: 0.5439 - val_class_label_accuracy: 0.1189
Epoch 2/25
121/121 [==============================] - 22s 183ms/step - loss: 3.2239 - box_coodinates_loss: 5.0554e-04 - class_label_loss: 3.2234 - box_coodinates_accuracy: 0.5537 - class_label_accuracy: 0.1444 - val_loss: 2.9736 - val_box_coodinates_loss: 4.8573e-04 - val_class_label_loss: 2.9731 - val_box_coodinates_accuracy: 0.5323 - val_class_label_accuracy: 0.2855
Epoch 3/25
121/121 [==============================] - 23s 187ms/step - loss: 2.9129 - box_coodinates_loss: 4.1446e-04 - class_label_loss: 2.9125 - box_coodinates_accuracy: 0.5649 - class_label_accuracy: 0.2140 - val_loss: 2.6990 - val_box_coodinates_loss: 4.2898

In [19]:
model.save(MODEL_PATH)